In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
import fileinput

In [41]:
quartiles = 'q1|q2|q3|q4' #arg
quartiles = quartiles.lower()
drop_without_quartile = False if 'q4' in quartiles else True

In [42]:
core_df = pd.read_excel('./Data/Source Datasets/Томск.xlsx', na_values=['Q-нет', ' ']). \
loc[:, ['Авторы', 'Квартиль', 'Название публикации']]
print(len(core_df))
core_df = core_df.loc[core_df.loc[:, 'Авторы'].notnull(), :]
print(len(core_df))
core_df = core_df.loc[core_df.loc[:, 'Квартиль'].str. \
                      lower().str. \
                      contains(quartiles, na=(not drop_without_quartile)) , :]
core_df = core_df.loc[:, ['Авторы', 'Название публикации']]
print(len(core_df))
core_df.head()

26140
26134
26134


,Авторы,Название публикации
0,"Saltykova, O.A.",On the contact interaction of a two-layer beam...
1,"Krysko, A.V.",On the contact interaction of a two-layer beam...
2,"Pestryakov, A.",Novel route of synthesis of ultra-small Au nan...
3,"Sivkov, A.",Boron carbide B4C ceramics with enhanced physi...
4,"Rakhmatullin, I.",Boron carbide B4C ceramics with enhanced physi...


has_scopus_name = lambda row: False if row['Издание'].find('Scopus') == -1 else True
core_df['Scopus'] = core_df.apply(has_scopus_name, axis=1)
core_df = core_df[core_df['Scopus']]

row = core_df.loc[a, :].iloc[0]
row

def split(stucked_str, delimiter):
    srs = pd.Series(stucked_str.split(delimiter))
    srs = srs.str.strip()
    return srs

names = split((row['Авторы'] + ', JR., kjjk.f'), ',')
names[~names.str.contains('JR.')]

authors = {}
authors_pairs = {}

names = split(row['Авторы'], ',')
if (any(names.str.contains('JR', case=False) == True)): #what does 'JR.' after name means? skip those
    print("FUUUUuuUUUUUUUUU")
ids = split(row['ids'], ';')
ids = ids[ids.str.isnumeric()] # filter empty strings

names = split(row['Авторы'], ',')
if (any(names.str.contains('JR', case=False) == True)): #what does 'JR.' after name means? skip those
    print("FUUUUuuUUUUUUUUU")
ids = split(row['ids'], ';')
ids = ids[ids.str.isnumeric()] # filter empty strings

for i in range(len(names)):
    authors.setdefault(ids[i], {'name':names[i], 'pub':0}) # add to dict 'authors' [id] -> (name, 0 {pulications})
    authors[ids[i]]['pub'] += 1

ids_pairs = combinations(ids, 2)
for pair in ids_pairs:
    pair = tuple(sorted(pair))
    if pair[0] == pair[1]:
        continue
    authors_pairs.setdefault(pair, 0)
    authors_pairs[pair] += 1

def process_row(row):
    names = split(row['Авторы'], ',')
    if (any((names.str.len() <= 3) == True)):   #what does names with less than 3 characters
                                                #mean? skip those
        return
    ids = split(row['ids'], ';')
    ids = ids[ids.str.isnumeric()] # filter empty strings
    if len(names) > len(ids):
        return True
        names = names[:len(ids)]
    if len(names) < len(ids):
        return True
        ids = ids[:len(names)]
    
    for i in range(len(names)):
        authors.setdefault(ids[i], {'name':names[i], 'count':0}) # add to dict 'authors' [id] -> (name, 0 {pulications})
        authors[ids[i]]['count'] += 1

    ids_pairs = combinations(ids, 2)
    for pair in ids_pairs:
        pair = tuple(sorted(pair))
        if pair[0] == pair[1]:
            continue
        authors_pairs.setdefault(pair, 0)
        authors_pairs[pair] += 1
    return

authors = {}
authors_pairs = {}

_ = core_df.apply(func=process_row, axis=1)

authors = pd.DataFrame.from_dict(authors, orient='index')
authors = authors.reset_index().rename(columns={'index':'id'}).set_index('id')
authors_pairs = pd.DataFrame.from_dict(authors_pairs, orient='index')

authors_pairs = authors_pairs.reset_index().rename(columns={0:'count'})

authors_pairs[['id1', 'id2']] = authors_pairs.loc[:, 'index'].apply(lambda pair: pd.Series([pair[0], pair[1]]))

authors_pairs = authors_pairs.loc[:, ['count', 'id1', 'id2']]

print(authors.head())
print(authors_pairs.head())

In [43]:
author_count = core_df.loc[:, ['Авторы', 'Название публикации']]. \
groupby(by=['Авторы']).count(). \
rename(columns={'Название публикации':'Публикаций'})
print(len(author_count))
author_count.head()

7811


,Публикаций
Авторы,
"Abakumov, A.",1
"Abashkin, A.",2
"Abbasi, M.",2
"Abdirasul Uulu, T.",1
"Abdousaliamov, A.V.",1


author_count = author_count.merge(core_df.loc[:, ['Авторы', 'AuthorID']].drop_duplicates('AuthorID'),
                   left_index=True, right_on='AuthorID',
                   validate='one_to_one')
print(len(author_count))
author_count.head()

In [44]:
group_pubs = core_df.loc[:, ['Авторы', 'Название публикации']]. \
groupby(by=['Название публикации'])

pairs_pub = {}
def process_group_of_authors(df):
    ids = df.loc[:, 'Авторы']
    ids_pairs = combinations(ids, 2)
    for pair in ids_pairs:
        pair = tuple(sorted(pair))
        if pair[0] == pair[1]:
            continue
        pairs_pub.setdefault(pair, 0)
        pairs_pub[pair] += 1
        
group_pubs.apply(process_group_of_authors)

""


In [45]:
pairs_count = pd.DataFrame.from_dict(pairs_pub, orient='index', columns=['count'])
pairs_count = pairs_count.reset_index()
pairs_count[['id1', 'id2']] = pairs_count.loc[:, 'index'].apply(pd.Series)
pairs_count.drop(columns=['index'], inplace=True)
pairs_count.head()

,count,id1,id2
0,1,"Evdokimov, K.E.","Melik-Gaykazyan, M.V."
1,5,"Goponenko, A.S.","Stukach, O.V."
2,4,"Goponenko, A.S.","Kochumeev, V.A."
3,6,"Kochumeev, V.A.","Stukach, O.V."
4,2,"Bilyalov, A.A.","Gavrilenko, M.A."


authors.head()

authors = authors.reset_index()

In [46]:
author_count = author_count.reset_index()

In [47]:
author_count.head()

,Авторы,Публикаций
0,"Abakumov, A.",1
1,"Abashkin, A.",2
2,"Abbasi, M.",2
3,"Abdirasul Uulu, T.",1
4,"Abdousaliamov, A.V.",1


In [48]:
pairs_count.head()

,count,id1,id2
0,1,"Evdokimov, K.E.","Melik-Gaykazyan, M.V."
1,5,"Goponenko, A.S.","Stukach, O.V."
2,4,"Goponenko, A.S.","Kochumeev, V.A."
3,6,"Kochumeev, V.A.","Stukach, O.V."
4,2,"Bilyalov, A.A.","Gavrilenko, M.A."


In [49]:
#convert dataset to vis.js format
edges = pairs_count.rename(columns={'count':'value', 'id1':'from', 'id2':'to'})
edges.loc[:, 'title'] = 'Совместных работ: ' + edges.loc[:, 'value'].astype(str)

nodes = author_count.rename(columns={'Авторы':'label', 'Публикаций':'mass'}) #title
nodes.loc[:, 'id'] = nodes.loc[:, 'label']
nodes.loc[:, 'value'] = np.sqrt(nodes.loc[:, 'mass'] / np.pi)
nodes.loc[:, 'title'] = 'Публикаций: ' + nodes.loc[:, 'mass'].astype(str)
nodes.loc[:, 'group'] = np.random.randint(1, 9, nodes.shape[0])
print(nodes.head())
print(len(nodes))
print(edges.head())
print(len(edges))

                 label  mass                   id     value          title  \
0         Abakumov, A.     1         Abakumov, A.  0.564190  Публикаций: 1   
1         Abashkin, A.     2         Abashkin, A.  0.797885  Публикаций: 2   
2           Abbasi, M.     2           Abbasi, M.  0.797885  Публикаций: 2   
3   Abdirasul Uulu, T.     1   Abdirasul Uulu, T.  0.564190  Публикаций: 1   
4  Abdousaliamov, A.V.     1  Abdousaliamov, A.V.  0.564190  Публикаций: 1   

   group  
0      5  
1      5  
2      5  
3      8  
4      6  
7811
   value             from                     to                title
0      1  Evdokimov, K.E.  Melik-Gaykazyan, M.V.  Совместных работ: 1
1      5  Goponenko, A.S.          Stukach, O.V.  Совместных работ: 5
2      4  Goponenko, A.S.        Kochumeev, V.A.  Совместных работ: 4
3      6  Kochumeev, V.A.          Stukach, O.V.  Совместных работ: 6
4      2   Bilyalov, A.A.       Gavrilenko, M.A.  Совместных работ: 2
18706


In [50]:
# required for faster physics calculation
edges.loc[:, 'hidden'] = True
print(edges.head())

   value             from                     to                title  hidden
0      1  Evdokimov, K.E.  Melik-Gaykazyan, M.V.  Совместных работ: 1    True
1      5  Goponenko, A.S.          Stukach, O.V.  Совместных работ: 5    True
2      4  Goponenko, A.S.        Kochumeev, V.A.  Совместных работ: 4    True
3      6  Kochumeev, V.A.          Stukach, O.V.  Совместных работ: 6    True
4      2   Bilyalov, A.A.       Gavrilenko, M.A.  Совместных работ: 2    True


In [51]:
def replace_str(filename, text_to_search, replacement_text):
    with fileinput.FileInput(filename, inplace=True) as file:
        for line in file:
            print(line.replace(text_to_search, replacement_text), end='')

In [52]:
edges_filename = 'edges_Томск ' + quartiles + '.json'
nodes_filename = 'nodes_Томск ' + quartiles + '.json'

### Запись в файл (без координат)

In [53]:
# Вывод в файл

edges.to_json(edges_filename, orient='records', force_ascii=True)
nodes.to_json(nodes_filename, orient='records', force_ascii=True)

replace_str(nodes_filename, '"title"', ' title ')
replace_str(nodes_filename, '"value"', ' value ')
replace_str(nodes_filename, '"group"', ' group ')
replace_str(nodes_filename, '"id"', ' id ')
replace_str(nodes_filename, '"label"', ' label ')
replace_str(nodes_filename, '"mass"', ' mass ')
replace_str(nodes_filename, '"x"', ' x ')
replace_str(nodes_filename, '"y"', ' y ')
# replace_str('pers_pub_net.json', '"physics"', ' physics ')
# replace_str('pers_pub_net.json', '"false"', ' false ')

replace_str(edges_filename, '"value"', ' value ')
replace_str(edges_filename, '"from"', ' from ')
replace_str(edges_filename, '"title"', ' title ')
replace_str(edges_filename, '"to"', ' to ')
replace_str(edges_filename, '"hidden"', ' hidden ')

## setting up positions of nodes

In [ ]:
positions = pd.read_json('Томск_positions_' + quartiles + '.json', orient='records')
print(len(positions))
positions.head()

In [ ]:
positions.loc[:, 'id'] = positions.loc[:, 'id'].astype(int)
nodes.loc[:, 'id'] = nodes.loc[:, 'id'].astype(int)

In [ ]:
nodes_with_coord = nodes.merge(right=positions, left_on='id', right_on='id', validate='one_to_one')
nodes_with_coord = nodes_with_coord.sort_values('mass', ascending=False)

In [ ]:
nodes_with_coord.head()

In [ ]:
edges.to_json(edges_filename, orient='records', force_ascii=True)
nodes_with_coord.to_json(nodes_filename, orient='records', force_ascii=True)

replace_str(nodes_filename, '"title"', ' title ')
replace_str(nodes_filename, '"value"', ' value ')
replace_str(nodes_filename, '"group"', ' group ')
replace_str(nodes_filename, '"id"', ' id ')
replace_str(nodes_filename, '"label"', ' label ')
replace_str(nodes_filename, '"mass"', ' mass ')
replace_str(nodes_filename, '"x"', ' x ')
replace_str(nodes_filename, '"y"', ' y ')
# replace_str('pers_pub_net.json', '"physics"', ' physics ')
# replace_str('pers_pub_net.json', '"false"', ' false ')

replace_str(edges_filename, '"value"', ' value ')
replace_str(edges_filename, '"from"', ' from ')
replace_str(edges_filename, '"title"', ' title ')
replace_str(edges_filename, '"to"', ' to ')
replace_str(edges_filename, '"hidden"', ' hidden ')

stucked_names = core_df.loc[:, 'Авторы']
stucked_names.head()

def split_names(stucked_names_entry):
    splitted = stucked_names_entry.split('\n')
    str_list = []
    for substr in splitted:
        temp = ' '.join(substr.split())
        str_list.append(temp)
    str_list = list(filter(None, str_list))
    return str_list

def pairwise(t):
    it = iter(t)
    return zip(it,it)

def detect_faculty(str_with_faculty):
    selector = {
        str_with_faculty.find('МТ'): 'МТ',
        str_with_faculty.find('ИУ'): 'ИУ',
        str_with_faculty.find('РЛ'): 'РЛ',
        str_with_faculty.find('ФН'): 'ФН',
        str_with_faculty.find('СМ'): 'СМ',
        str_with_faculty.find('Э'): 'Э',
        str_with_faculty.find('РК'): 'РК',
        str_with_faculty.find('БМТ'): 'БМТ',
        str_with_faculty.find('Л'): 'Л',
        str_with_faculty.find('ИБМ'): 'ИБМ',
        str_with_faculty.find('СГН'): 'СГН',
        str_with_faculty.find('РКТ'): 'РКТ',
        str_with_faculty.find('АК'): 'АК',
        str_with_faculty.find('ПС'): 'ПС',
        str_with_faculty.find('РТ'): 'РТ',
        str_with_faculty.find('ОЭП'): 'ОЭП',
        str_with_faculty.find('ЮР'): 'ЮР',
        str_with_faculty.find('ГУИМЦ'): 'ГУИМЦ',
        str_with_faculty.find('ФМОП'): 'ФМОП',
        str_with_faculty.find('ФВО'): 'ФВО',
    }
    try:
        sel = selector[True]
        assert(type(sel) == str), 'Multiple faculties detected: {}'.format(type(sel))
        return sel
    except KeyError as e:
        return 'Other'

def process_group_of_names_df(group_of_names, personal_publications_df, pair_publications_df):
    splitted = split_names(group_of_names)
    name_position_pairs = pairwise(splitted)
    bmstu_members = [entry for entry in name_position_pairs if (entry[1].find('Не МГТУ') == -1)]
    
    
    
def process_group_of_names(group_of_names):
    splitted = split_names(group_of_names)
    name_position_pairs = pairwise(splitted)
    bmstu_members = [entry for entry in name_position_pairs if (entry[1].find('Не МГТУ') == -1)]
    
    for entry in bmstu_members:
        faculty = detect_faculty(entry[1])
        personal_publications_num.setdefault(entry[0], [0, faculty])
        personal_publications_num[entry[0]][0] += 1
    for single_comb in combinations(bmstu_members, 2):
        sorted_comb = sorted(single_comb, key=lambda tup: tup[0])
        names_comb = (sorted_comb[0][0], sorted_comb[1][0])
        pair_publications_num.setdefault(names_comb, 0)
        pair_publications_num[names_comb] += 1

personal_publications_num = {}
pair_publications_num = {}
stucked_names = core_df['Авторы']
stucked_names.apply(process_group_of_names);

In [ ]:
personal_publications_num_df = pd.DataFrame.from_dict(personal_publications_num, orient='index',
                                                      columns=['pub_num', 'faculty'])
personal_publications_num_df.index.names = ['full_name']
personal_publications_num_df.reset_index(inplace=True)
personal_publications_num_df.loc[:, 'id'] = personal_publications_num_df.apply(lambda row: id(row['full_name']),
                                                                               axis=1)


pair_publications_num_df = pd.DataFrame.from_dict(pair_publications_num, orient='index', columns=['pair_pub_num'])
pair_publications_num_df.index.names = ['names_pair']
pair_publications_num_df.reset_index(inplace=True)
get_ids = lambda row: pd.Series([id(row['names_pair'][0]),
                                 id(row['names_pair'][1])])
pair_publications_num_df[['name1_id', 'name2_id']] = pair_publications_num_df.apply(get_ids, axis=1)

def append_key_data(row):
     return row.loc['full_name'] + ' (' + str(row.loc['pub_num']) + ', ' + row.loc['faculty'] + ')'

personal_publications_num_df['key_data'] = personal_publications_num_df.apply(
    append_key_data, axis=1)

personal_publications_num_network_format_df = personal_publications_num_df.rename(
    columns={'key_data':'title', 'pub_num':'value', 'faculty':'group'})[['title', 'value', 'group', 'id']]
pair_publications_num_network_format_df = pair_publications_num_df.rename(
    columns={'pair_pub_num':'value', 'name1_id':'from', 'name2_id':'to'})[['value', 'from', 'to']]

personal_publications_num_network_format_df.to_json('pers_pub_net.json', orient='records')
pair_publications_num_network_format_df.to_json('pair_pub_net.json', orient='records')